# Compressed Sensing (CS) based ECG compressor

## Table of Contents

TO BE WRITTEN

# Goal of Project

TO BE WRITTEN

Roadmap:
- reproducing idea from paper bla bla bla
- in general: study ("emulate", not really) best solution for a CS-based compressor for ECG to be used with remote-ECG-devices, small, limited storage capability, limited computational power:
    - __phase 1__ compute dictionary $\Psi$ and measurement matrix $\Phi$ before actually using the device to measure the patient ecg
    - __phase 2__ pass $\Psi$, $\Phi$ to the device, take __already compressed measurements__ $y$ (we'll see that this is core idea of CS)
    - __phase 3__ store only $y$, $\Psi$, $\Phi$ and send them back to _more computationally powerful system_ where recovery happens
- paper focuses also on how such hardware is built, we will be more generic
- exploit data from Physionet.org exactly like the paper did
- test different dictionaries, both _fixed dictionaries_ (_DCT_, _DWT_, _KL_) and _adaptive dictionary learning_ (_MOM_, _K-SVD_)
- test how dimension of measurement matrix $\Phi$ is related to processing speed in __phase 2__
- test different _recovery methods_ ("classic" _l1-minimization_, _LASSO_, _Greedy Algorithms_, _Smooth-L0_, _Baisis Pursuit_), always use newly developed __Kronecker technique__ 
- testing robustness to noise with _additive noise_



# Theoretical Review

## Theory: sparsity and compression

## Theory: metrics and choices

TO BE WRITTEN

Roadmap:
- also here choices strongly based on paper bla bla bla
- _Compression rate (CR)_ of 75%, obtained directly "by construction" in the process (through _measurement matrix_)
- _complexity of algorithm in recording_ directly bind to _power consumption_, very relevant for remote-ecg-recorders
- _emergency ecg_ , such in ambulatory require _processing speed_, both during sampling and in reconstruction
    - both previous point are related to _optimal choice of measurement matrix_, as it will be explained later
- _accuracy_ through quantitive metrics: _PRD_, _SNR_
    - especially important to evaulate different dictionaries adn different methods of reconstruction

## Theory: measurement matrix

TO BE WRITTEN

Roadmap:
- Relation between $\Phi$ dimension and speed of "sensing phase" (phase 2)
- Relation between randomicity and RIP property

## Theory: dictionaries

### _fixed dictionaries_ vs _adaptive dictionary learning_
TO BE WRITTEN

---

### Fixed dictionaries

#### DCT
TO BE WRITTEN

#### DWT
TO BE WRITTEN

#### KL
TO BE WRITTEN

---

### Adaptive Dictionary Learning
TO BE WRITTEN

#### MOM
TO BE WRITTEN

#### K-SVD
TO BE WRITTEN

## Theory: reconstruction of the signal

### Reconstruction methods

(we already explained the maths before, here is practival parts)

#### "classic" _l1-minimization_ problem
TO BE WRITTEN

#### LASSO
TO BE WRITTEN

#### Greedy Algorithms
TO BE WRITTEN

#### Smooth-L0
TO BE WRITTEN

#### Basis Pursuit
TO BE WRITTEN

---

### Kronecker Techinque

TO BE WRITTEN

# Code implementation (python)


---
__Pre-sampling phase__

- Generate dictionary $\Psi$
- Generate Measurement Matrix $\Phi$

---
__Sampling phase__
- $y = \Phi \Psi s$, simulate what would happen on device (_compute block by block, later concatenate results_)

---
__Recovery phase__
- Reconstruct signal

---
__Evaluate result__
1. Sampling faster for smaller $\Phi$?
2. Which dictionary are the best?
3. Which recovery method is best?
4. Robustness to noise?

---

## Code: dictionary

- First step of __Pre-sampling phase__
- Develop code that outputs a dictionary $\Psi \in \mathbb{R}^{d \times d}$
- $d$ will be the block dimension. 
- Code will be able to create the dictionary with one of the methods
    - Fixed dictionaries: DCT, DWT, KL
    - Adaptive Dictionary Learning: MOM, K-SVD



In [ ]:
import numpy as np
import pywt

def create_wavelet_basis(N, wavelet_name='db4'):
    """
    Create the wavelet basis matrix for a given signal dimension and wavelet type.

    Parameters:
    N (int): The dimension of the signal.
    wavelet_name (str): The name of the wavelet (default is 'db4').

    Returns:
    waveletBasis (ndarray): The combined basis matrix, where the first N/2 rows are the
                            approximation basis and the second N/2 rows are the detail basis.
    """
    # Load the specified wavelet
    wavelet = pywt.Wavelet(wavelet_name)
    
    # Extract the scaling (low-pass) and wavelet (high-pass) filter coefficients
    h = wavelet.dec_lo
    g = wavelet.dec_hi
    
    # Length of the filter
    L = len(h)
    
    # Initialize matrices to store the basis vectors
    Phi = np.zeros((N//2, N))
    Psi = np.zeros((N//2, N))
    
    # Construct the approximation (Phi) and detail (Psi) basis matrices
    for i in range(N//2):
        Phi[i, i*2:i*2+L] = h
        Psi[i, i*2:i*2+L] = g
    
    # Combine Phi and Psi into a single matrix
    waveletBasis = np.vstack((Phi, Psi))
    
    return waveletBasis




## Code: measurement matrix

- Second step of __Pre-sampling phase__
- Simple code to produce a $\Phi \in \mathbb{R}^{d/4 \times d}$ measurement matrix
- Coefficient of $\Phi$ will be random (_Bernoulli_) extraction of $-1,1$

## Code: sampling phase

- Only step of __Sampling phase__ 
- Compute $y_{j} = \Phi \Psi s_{j}$ on each $j$-th block of signal of dimension $d$
- Progressively attach new $y_{j}$ to previous unitil you obtain $y$ _compressed measurement_ (of whole signal $s$)

## Code: recovery

- Only step of __Recovery phase__
- Code that will be able to use _one of the selected methods_ to reconstruct an apporoximation of original signal $s$ from the _compressed measurement_ $y$
    - $l1$-minimization
    - LASSO
    - Greedy
    - Smooth-L0
    - Basis Pursuit
- __All methods (if possible) must exploit Kronecker Technique__

## Data: MIT–BIH Arrhythmia Database

USE THE ONE IN THE PREVIOUS NOTEBOOK

## Code: test for best dictionary

- Test all dictionaries to show with big number of data which are the one to perform better (we expect DWT to be the best of fixed, and that adaptive are better than fixed in general)
- __#Records:__ Test on __MULTIPLE patients__ records is a MUST, especially to show that adaptive are better 
- __#Dictionaries:__ Test all dictionaries, that's what we are doing ...
- __#ReconstructionMethods:__ Test with __only one reconstruction method__

## Code: test for best reconstruction method

- Test to find which recontruction method is the best
- __#Records:__ Test on a __single patient__ record should be fine 
- __#Dictionaries:__ Test with __a single dictionary type__
- __#ReconstructionMethods:__ Test with __all reconstruction methods__

## Code: test for correct block dimension

- Test to show that __sampling phase process speed is inversly proportional to block dimension__
- __#Records:__ Test on a __single patient__ record should be fine 
- __#Dictionaries:__ Test with __a single dictionary type__ (USE BEST!)
- __#ReconstructionMethods:__ Test with __only one reconstruction method__ (USE BEST!)

## Code: test for noise robustness

- Test if robust to noise by adding noise to the signal
- __#Records:__ Test on a __single patient__ record should be fine 
- __#Dictionaries:__ Test with __a single dictionary type__ (USE BEST!)
- __#ReconstructionMethods:__ Test with __only one reconstruction method__ (USE BEST!)

# Conclusions

TO BE WRITTEN

# References

TO BE WRITTEN

# Appendix

TO BE WRITTEN?

For instance explanation of what ecg is